In [1]:
from dotenv import load_dotenv
import os

from typing import TypedDict, Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langgraph.prebuilt import ToolNode, InjectedState
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.types import Command
from langchain_core.tools import tool, InjectedToolCallId


from gen_test_cases import generate_test_cases_from_requirements
from gen_requirements import generate_requirement_from_doc 

load_dotenv()
llm_api_key = os.getenv("GOOGLE_API_KEY")

In [2]:

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens = None,
    api_key = llm_api_key,
)

In [ ]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    project_name: str
    summary: str
    description: str
    requirements: list
    testCases: list


In [4]:
@tool
def generate_requirements_from_document_pdf_tool(
    x: Annotated[str, "base64 encoded string representation of a pdf file"],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Creating a brief summary of the document and a list of requirements from the project document in base64 encoded string representation of a pdf file """
    requirements, summary = generate_requirement_from_doc(x)
    return Command(update={
        "summary": summary,
        "requirements": requirements,
        "messages": [
            ToolMessage(
                "Successfully created requirements from the document",
                tool_call_id=tool_call_id
            )
        ]
    })


In [5]:
@tool
def generate_testCases_fromRequirements_tool(
    state: Annotated[AgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """Creating a list of test cases from a list of requirements provided with test case"""
    requirements = state.get('requirements')
    context = state.get('summary')
    if not requirements or not context:
        return Command(update={
            "messages": [
            ToolMessage(
                "Requirements or Context missing",
                tool_call_id=tool_call_id
            )
        ]
    })
    testCases_dict = generate_test_cases_from_requirements(requirements, context)
    return Command(update={
        "testCases": testCases_dict,
        "messages": [
            ToolMessage(
                "Successfully created test cases from the requirements",
                tool_call_id=tool_call_id
            )
        ]
    })

In [6]:

tools = [generate_testCases_fromRequirements_tool,generate_requirements_from_document_pdf_tool]
llm_with_tools = llm.bind_tools(tools)

In [ ]:

def call_model(state: AgentState):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [8]:
def custom_tool_condition(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools_node"
    return "call_model"

In [9]:
builder = StateGraph(AgentState)
builder.add_node("call_model", call_model)
builder.add_node("tools_node", ToolNode(tools))

builder.add_edge(START,"call_model")
builder.add_conditional_edges(
    "call_model",
    custom_tool_condition,
)
builder.add_edge("tools_node","call_model")
builder.add_edge("call_model", END)

graph = builder.compile()

In [ ]:
#Initialize state by loading a project and its info (if any)
currState = AgentState(messages=[SystemMessage(content="You are a helpful assistant tasked with performing QA in SDLC.")],
                       project_name="QA_automation")
# Get user Input
userInputContent = "hello"

# Actually run
currState = {"messages":currState["messages"]+[(HumanMessage(content=userInputContent))]}
print(currState)
graph.invoke(currState)

{'messages': [SystemMessage(content='You are a helpful assistant tasked with performing QA in SDLC.', additional_kwargs={}, response_metadata={}), HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]}
